In [11]:
# import openai
# import time 
import matplotlib.pyplot as plt
import numpy as np
# import json
import pandas as pd
import re
# %matplotlib inlinew2v_model
# from rake_nltk import Rake
# import multiprocessing
from gensim.models.phrases import Phrases, Phraser
# from gensim.models import Word2Vec
# import seaborn as sns
# sns.set_style("darkgrid")

# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# import spacy
import gensim
# from collections import defaultdict
# from tqdm import tqdm
from thefuzz import fuzz

In [12]:
df_arx = pd.read_csv('arxiv_stop.csv',names=["id","abstract","date", "s_indc"])
ab_arr = df_arx["abstract"].to_numpy()
df_arx["date"] = pd.to_datetime(df_arx["date"])
year_arr = df_arx['date'].dt.year.to_numpy()
month_arr = df_arx['date'].dt.month.to_numpy()

In [7]:
# np.save("year_arr.npy",year_arr)
# np.save("month_arr.npy",month_arr)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)

In [8]:
# # Abstracts that contain the word "Surprise"
# df_arx[df_arx["s_indc"]==1]["abstract"].to_numpy().shape

# # Check for certain behaviour
# found = False
# while found == False:
#     a = np.random.choice(ab_arr)
#     if "<num>" in a:
#         print(a)
#         found = True

In [13]:
np.random.choice(ab_arr)

'   we demonstrate a technique to lock simultaneously two laser frequency to each step of a two_photon transition in the presence of a magnetic field sufficiently large to gain access to the hyperfine paschen_back regime a ladder configuration with the <num> and <num> term in a thermal vapour of <ltx> rb atom be use the two laser remain locked for more than <num> hour for the sum of the laser frequency which represent the stability of the two_photon lock we measure a frequency instability of less than the rb d <ltx> natural linewidth of <num> mhz for nearly all measure time scale'

Goal: Extract Concepts from the Abstracts. 
Naive Approach: Extract n-grams (max:4) and compare with known list of physics concepts.

In [14]:
def get_ngrams(sentences):
    """ Detects n-grams with n up to 4, and replaces those in the abstracts. """
    # Train a 2-word (bigram) phrase-detector
    bigram_phrases = gensim.models.phrases.Phrases(sentences,min_count=10,threshold=15)
    
    # And construct a phraser from that (an object that will take a sentence
    # and replace in it the bigrams that it knows by single objects)
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    
    # Repeat that for trigrams; the input now are the bigrammed-titles
    ngram_phrases = gensim.models.phrases.Phrases(bigram[sentences],min_count=10,threshold=15)
    ngram         = gensim.models.phrases.Phraser(ngram_phrases)
    
    # !! If you want to have more than 4-grams, just repeat the structure of the
    #    above two lines. That is, train another Phrases on the ngram_phrases[titles],
    #    that will get you up to 8-grams. 
    
    # Now that we have phrasers for bi- and trigrams, let's analyze them
    # The phrases.export_phrases(x) function returns pairs of phrases and their
    # certainty scores from x.
    bigram_info = {}
    for b, score in bigram_phrases.export_phrases().items():
        bigram_info[b] = [score, bigram_info.get(b,[0,0])[1] + 1]
        
    ngram_info = {}
    for b, score in ngram_phrases.export_phrases().items():
        ngram_info[b] = [score, ngram_info.get(b,[0,0])[1] + 1]
            
    # Return a list of 'n-grammed' abtracts, and the bigram and trigram info
    return [ngram[t] for t in sentences], bigram_info, ngram_info

sentences = [row.split() for row in ab_arr]
ngram_abstracts, bigrams, ngrams = get_ngrams(sentences)


In [15]:
ngrams

{'arise_from': [16.225993609640064, 1],
 'take_into': [79.73487204626626, 1],
 'take_into_account': [229.2777538159617, 1],
 'relationship_between': [85.05148452410589, 1],
 'truth_value': [114.43321722043586, 1],
 'arrive_at': [218.02097973995262, 1],
 'relative_entropy': [65.38743360911786, 1],
 'density_matrix': [109.40632540495238, 1],
 'per_unit': [140.32980313773066, 1],
 'intuitive_understanding': [69.47525308167211, 1],
 'fascinating_phenomenon': [15.80517572286434, 1],
 'basic_idea': [25.721509050894944, 1],
 'arise_naturally': [30.054771785077182, 1],
 'travel_wave': [39.655201470053214, 1],
 'lamb_dicke': [10301.0859082397, 1],
 'lamb_dicke_limit': [50.09136279843532, 1],
 'jaynes_cumming_model': [3317.3146291843905, 1],
 'rotate_wave': [71.68959394418584, 1],
 'rotate_wave_approximation': [54.94053429977102, 1],
 'non_rwa': [65.42647404275311, 1],
 'spin_dependent': [22.063069998147665, 1],
 'trap_ion': [72.18323555490852, 1],
 'difference_between': [17.902095988250146, 1],

In [12]:
# less than 4 
# et al 
# physics 

Process ngrams to improve quality of later predictions

In [13]:
# 1. analysiere ngrams 
# 2. save phys conc 
# 3. remove _ from non 

In [16]:
df_atomic = pd.read_csv('arxiv_atomic_concept.txt',names=["con"])
atomic_arr = np.array([con.replace(" ", "_") for con in df_atomic["con"].to_numpy()])

df_optic = pd.read_csv('arxiv_optics_concept.txt',names=["con"])
optic_arr = np.array([con.replace(" ", "_") for con in df_optic["con"].to_numpy()])

df_quantum = pd.read_csv('arxiv_quantum_concept.txt',names=["con"])
quantum_arr = np.array([con.replace(" ", "_") for con in df_quantum["con"].to_numpy()])

concept_compare_arr = np.unique(np.concatenate((atomic_arr,optic_arr,quantum_arr)))

naive preprecess

In [22]:

pre_selec_inx = np.array([c in ngrams for c in concept_compare_arr]).astype(int)

concept_pre_compare_arr = concept_compare_arr[pre_selec_inx==1]
concept_continue_compare_arr = concept_compare_arr[pre_selec_inx==0]
concept_continue_compare_arr.shape


# optic_selected_arr = optic_arr[optic_inx==1]
# quantum_selected_arr = quantum_arr[quantum_inx==1]
# selected_concept_arr = np.concatenate((atomic_selected_arr, optic_selected_arr, quantum_selected_arr))


(30703,)

In [23]:
def find_overlapping_concepts(concepts1, concepts2, threshold=80):
    overlapping_concepts = []
    non_overlapping_concepts = []

    for concept1 in concepts1:
        match = False 
        for concept2 in concepts2:
            # Use fuzz ratio to measure similarity
            
            similarity_ratio = fuzz.partial_token_sort_ratio(concept1, concept2)
           
            # Adjust the threshold based on your requirements
            if similarity_ratio >= threshold and len(concept1)>3:
                overlapping_concepts.append((concept1, concept2, similarity_ratio))
                match = True 
                break 
        if not match:
            non_overlapping_concepts.append(concept1) 

    return np.array(overlapping_concepts), np.array(non_overlapping_concepts)

# # Example usage:
concept_list1 = ["quantum_phase_transition", "electron", "other_concept"]
concept_list2 = ["phase_transition", "electron_hole", "another_concept"]

result = find_overlapping_concepts(concept_list1, concept_list2)
print("Overlapping Concepts:", result)


Overlapping Concepts: (array([['quantum_phase_transition', 'phase_transition', '81'],
       ['electron', 'electron_hole', '100']], dtype='<U24'), array(['other_concept'], dtype='<U13'))


In [25]:
check_arr = np.array(list(ngrams.keys()))[:5000]
overlapping_concepts, non_overlapping_concepts = find_overlapping_concepts(check_arr,concept_continue_compare_arr[:5000],threshold=80)
overlapping_concepts.shape, non_overlapping_concepts.shape

((2724, 3), (2276,))

In [26]:
for ab in ngram_abstracts:
    for rem_ngram in non_overlapping_concepts:
        if rem_ngram in ab:
            inx_arr = [index for index, content in enumerate(ab) if rem_ngram == content]
            if inx_arr != []:
                replace_by = rem_ngram.replace("_", " ")
                for index in inx_arr:
                    ab[index] = replace_by

In [ ]:
overlapping_concepts

In [28]:
np.save("overlapping_concepts.npy",np.concatenate((overlapping_concepts[:,0],concept_pre_compare_arr)))
np.save("ngram_abstracts.npy",[' '.join(ab) for ab in ngram_abstracts])

Sort n-grams by certainty to check result

In [24]:
sortedbis = sorted( [(bigrams[b][0], bigrams[b][1], b) for b in bigrams.keys()] )[::-1]
sortedns  = sorted( [(ngrams[b][0], ngrams[b][1], b) for b in ngrams.keys()] )[::-1]
print("Top bigrams by certainty:")
for i in range(20):
    print("{0:2}: {1:50} \t({2}) ".format(i+1, str(sortedbis[i][2]), "%.2f"%sortedbis[i][0]))
    
print("\n")
print("Top 3- and 4-grams by certainty:")

i = 0 
cnt = 0 
while cnt < 20 :
    if str(sortedns[i][2]).count("_")>1:
        print("{0:2}: {1:50} \t({2}) ".format(i+1, str(sortedns[i][2]), "%.2f"%sortedns[i][0]))
        cnt += 1
    i += 1

Top bigrams by certainty:
 1: duffin_kemmer_petiau                               	(46479895.00) 
 2: altland_zirnbauer                                  	(40417300.00) 
 3: falicov_kimball                                    	(28292110.00) 
 4: foldy_wouthuysen                                   	(26271245.00) 
 5: fenna_matthews_olson                               	(25597623.33) 
 6: clebsch_gordan                                     	(21825342.00) 
 7: bialynicki_birula                                  	(18187785.00) 
 8: autler_townes                                      	(16840541.67) 
 9: nakajima_zwanzig                                   	(16166920.00) 
10: runge_kutta                                        	(14819676.67) 
11: ornstein_uhlenbeck                                 	(12798811.67) 
12: reissner_nordstrom                                 	(12630406.25) 
13: ping_pong                                          	(12630406.25) 
14: retro_reflecte                                 

In [ ]:
def keep_words_with_underscore(input_string):
    # Define a regular expression pattern to match words with underscores
    pattern = r'\b\w*_[\w_]*\b'

    # Use re.findall to extract words that match the pattern
    matching_words = re.findall(pattern, input_string)

    # Join the matching words to form the final string
    result = ' '.join(matching_words)

    return result

